In [1]:
#! /usr/bin/python3
import json
import sys
import networkx as nx
import numpy as np
import pennylane as qml
from pennylane import qaoa
from matplotlib import pyplot as plt

In [22]:

# DO NOT MODIFY any of these parameters
NODES = 6
N_LAYERS = 10


def find_max_independent_set(graph, params):
    """Find the maximum independent set of an input graph given some optimized QAOA parameters.

    The code you write for this challenge should be completely contained within this function
    between the # QHACK # comment markers. You should create a device, set up the QAOA ansatz circuit
    and measure the probabilities of that circuit using the given optimized parameters. Your next
    step will be to analyze the probabilities and determine the maximum independent set of the
    graph. Return the maximum independent set as an ordered list of nodes.

    Args:
        graph (nx.Graph): A NetworkX graph
        params (np.ndarray): Optimized QAOA parameters of shape (2, 10)

    Returns:
        list[int]: the maximum independent set, specified as a list of nodes in ascending order
    """
    
    max_ind_set = []

    # QHACK #
#     nx.draw(graph, with_labels=True)
    #print(params)
    cost_h, mixer_h = qaoa.max_independent_set(graph, constrained=True)
    def qaoa_layer(gamma, alpha):
        qaoa.cost_layer(gamma, cost_h)
        qaoa.mixer_layer(alpha, mixer_h)
    wires = range(NODES)
    depth = N_LAYERS
    def circuit(params, **kwargs):
#         for w in wires:
#             qml.Hadamard(wires=w)
        qml.layer(qaoa_layer, depth, params[0], params[1])
    dev = qml.device("qulacs.simulator", wires=wires)
    @qml.qnode(dev)
    def probability_circuit(gamma, alpha):
        circuit([gamma, alpha])
        return qml.probs(wires=wires)
    
    probs = probability_circuit(params[0], params[1])
    tup = [[float(probs[i]), i] for i in range(len(probs))]
    def getKey(item):
        return item[0]
    tup.sort(key = getKey)
    ans = tup[-1][1]
    for i in range(NODES):
        if (ans & (1 << i)) != 0:
            max_ind_set.append(NODES - i - 1)
    max_ind_set.reverse()
#     plt.style.use("seaborn")
#     plt.bar(range(2 ** len(wires)), probs)
#     plt.show()
    
    # QHACK #

    return max_ind_set


with open("2.in") as file:
    graph_string = file.read()
    graph_data = json.loads(graph_string)

    params = np.array(graph_data.pop("params"))
    graph = nx.json_graph.adjacency_graph(graph_data)

    max_independent_set = find_max_independent_set(graph, params)

    print(max_independent_set)

with open("2.ans") as file:
    print(file.read())
# if __name__ == "__main__":
#     # DO NOT MODIFY anything in this code block

#     # Load and process input
#     graph_string = sys.stdin.read()
#     graph_data = json.loads(graph_string)

#     params = np.array(graph_data.pop("params"))
#     graph = nx.json_graph.adjacency_graph(graph_data)

#     max_independent_set = find_max_independent_set(graph, params)

#     print(max_independent_set)

[0, 1, 2]
[0, 1, 2]
